# Подключаемся к Google Bigquery прямо из Jupyter Notebook

Импортируем библиотеки

In [1]:
import pandas as pd
from google.cloud import bigquery

PROJECT_ID = 'data-explorer-xxxxx'

client = bigquery.Client(project = PROJECT_ID)

Пробуем простой запрос

In [6]:
QUERY = ('SELECT *  FROM `bigquery-public-data.covid19_usafacts.summary` LIMIT 10')# сам SQL запрос
query_job = client.query(QUERY)  # API запрос
rows = query_job.result()  # принимает результаты запроса

print("Started job: {}".format(query_job.job_id)) # выводит id джобы
for row in query_job:
    print(row) #  вывод полных результатов
    print("{}: \t{}".format(row.state, row.deaths)) # форматированный вывод выбранных столбцов

Started job: d6107399-ee2f-45a9-82c1-948d575866ec
AL: 	0
AL: 	0
AL: 	0
AL: 	0
AL: 	0
AL: 	0
AL: 	0
AL: 	0
AL: 	0
AL: 	0


Подключаем магию и пишем SQL, как у себя в консоли

In [8]:
%load_ext google.cloud.bigquery

In [13]:
%%bigquery
#SELECT *  FROM `bigquery-public-data.covid19_usafacts.summary` LIMIT 15
SELECT state, EXTRACT(MONTH FROM date) month, SUM(confirmed_cases) cases, SUM(deaths) dth
FROM `bigquery-public-data.covid19_usafacts.summary`
GROUP BY state, month
ORDER BY dth DESC
LIMIT 15

,state,month,cases,dth
0,NY,10,14942897.000000000,1024871.000000000
1,NY,11,17023469.000000000,1010582.000000000
2,NY,8,13185142.000000000,1008124.000000000
3,NY,7,12546132.000000000,995344.000000000
4,NY,9,13396227.000000000,982588.000000000
5,NY,6,11515526.000000000,917920.000000000
6,NY,5,10695430.000000000,854223.000000000
7,TX,11,31313614.000000000,589995.000000000
8,CA,11,31639419.000000000,551470.000000000
9,FL,11,26828158.000000000,528311.000000000


Загружаем рузультаты запроса сразу в Pandas

In [16]:
QUERY = ('SELECT *  FROM `bigquery-public-data.covid19_usafacts.summary` LIMIT 1000')
df = client.query(QUERY).to_dataframe()

Смотрим на датафрейм

In [18]:
df.shape

(1000, 7)

In [19]:
df.sample(15)

,county_fips_code,county_name,state,state_fips_code,date,confirmed_cases,deaths
569,01001,Autauga County,AL,01,2020-09-25,1757.000000000,25.000000000
8,00000,Statewide Unallocated,AL,01,2020-01-30,0E-9,0E-9
770,01003,Baldwin County,AL,01,2020-05-27,281.000000000,9.000000000
437,01001,Autauga County,AL,01,2020-05-16,110.000000000,3.000000000
183,00000,Statewide Unallocated,AL,01,2020-07-23,0E-9,0E-9
946,01003,Baldwin County,AL,01,2020-11-19,7933.000000000,84.000000000
115,00000,Statewide Unallocated,AL,01,2020-05-16,0E-9,0E-9
89,00000,Statewide Unallocated,AL,01,2020-04-20,0E-9,0E-9
288,00000,Statewide Unallocated,AL,01,2020-11-05,0E-9,0E-9
432,01001,Autauga County,AL,01,2020-05-11,84.000000000,3.000000000
